In [1]:
import os
import sys
module_path_cobras = os.path.abspath(os.path.join('../../COBRAS+_Aras/cobras-plus-shared-master(1)/cobras-plus-shared-master'))
module_path_testing = os.path.abspath(os.path.join('../../COBRAS+_Aras/cobras-plus-shared-master(1)/COBRAS_testing'))

if module_path_cobras not in sys.path:
    sys.path.append(module_path_cobras)
    print("module path of cobras+ added")

if module_path_testing not in sys.path:
    sys.path.append(module_path_testing)
    print("module path of testing added")

module path of cobras+ added
module path of testing added


In [2]:
from cobras_ts.cobras_kmeans import COBRAS_kmeans
from cobras_ts.labelquerier import LabelQuerier

from datasets import Dataset
from before_clustering.generate_folds import generate_folds_for_dataset
from config import FOLD_RESULT_DIR, FIGURE_DIR
from pathlib import Path
from generate_clusterings.clustering_task import make_n_run_10_fold_cross_validation
from run_locally.run_tests import run_clustering_tasks_locally
from evaluate_clusterings.calculate_aris import calculate_n_times_n_fold_aris_for_testnames
from evaluate_clusterings.calculate_average_aris import calculate_average_aris
from evaluate_clusterings.calculate_aligned_rank import calculate_and_write_aligned_rank
from present_results.plot_aris import plot_average_ARI_per_dataset
from present_results.plot_aris import plot_overall_average_ARI
from present_results.plot_aligned_rank import plot_rank_comparison_file


In [3]:
if not Path(FOLD_RESULT_DIR).exists():
    generate_folds_for_dataset()

In [4]:
ds = Dataset("iris")
querier = LabelQuerier(ds.target)
clusterer_1 = COBRAS_kmeans(ds.data, querier, 100, store_intermediate_results=True,
                            compatibility_intermediate_results=True)
test_name_1 = "Code_aras_baseline"
#clusterer_1.cluster()

In [5]:
all_dataset_names = Dataset.get_standard_dataset_names()#["iris"]#
#all_dataset_names.remove('faces_eyes_imagenet')
clustering_tasks = make_n_run_10_fold_cross_validation(test_name_1, clusterer_1, querier, all_dataset_names, 3)


In [6]:
clusterer_2 = COBRAS_kmeans(ds.data, querier, 100, store_intermediate_results_extended=True, 
                            compatibility_intermediate_results=True,
                            check_ml_constraints=2,
                            check_all_instances=0,
                            check_all_instances_threshold=0.75)
test_name_2 = "Code_aras_params_2_0_0.75"
clustering_tasks.extend( make_n_run_10_fold_cross_validation(test_name_2, clusterer_2, querier, all_dataset_names, 3))

In [7]:
clusterer_3 = COBRAS_kmeans(ds.data, querier, 100, store_intermediate_results_extended=True, 
                            compatibility_intermediate_results=True,
                            check_ml_constraints=2,
                            check_all_instances=0.5,
                            check_all_instances_threshold=0.75)
test_name_3 = "Code_Aras_params_2_0.5_0.75"
clustering_tasks.extend( make_n_run_10_fold_cross_validation(test_name_3, clusterer_3, querier, all_dataset_names, 3))

In [8]:
clusterer_4 = COBRAS_kmeans(ds.data, querier, 100, store_intermediate_results_extended=True, 
                            compatibility_intermediate_results=True,
                            check_ml_constraints=2,
                            check_all_instances=0.5,
                            check_all_instances_threshold=0.75,
                           incremental=0.5)
test_name_4 = "Code_Aras_params_2_0.5_0.75_0.5"
clustering_tasks.extend( make_n_run_10_fold_cross_validation(test_name_4, clusterer_4, querier, all_dataset_names, 3))

In [9]:
run_clustering_tasks_locally(clustering_tasks,4)

100%|██████████| 630/630 [30:51<00:00,  2.94s/it]  


In [4]:
test_name_1 = "Code_aras_baseline"
test_name_2 = "Code_aras_params_2_0_0.75"
test_name_3 = "Code_Aras_params_2_0.5_0.75"
test_name_4 = "Code_Aras_params_2_0.5_0.75_0.5"
test_name_5 = "standard_COBRAS"
test_names = [test_name_1, test_name_5]#test_name_1,test_name_6,
comparison_name = 'Code_aras_test_10'
nb_of_cores = 4
query_budget = 100

#print(clusterer_1.logger.nr_reused_constraints)

calculate_n_times_n_fold_aris_for_testnames(test_names, nb_cores=nb_of_cores)
calculate_average_aris(test_names, query_budget)
calculate_and_write_aligned_rank(test_names,comparison_name)

plot_average_ARI_per_dataset(comparison_name, test_names, test_names)
plot_overall_average_ARI(comparison_name, test_names, test_names)
plot_rank_comparison_file(comparison_name, test_names, test_names)

#calculate_and_plot_query_reuse(comparison_name, test_names)
#calculate_and_plot_query_reuse_per_dataset(comparison_name, test_names)

print(f"all resulting plots are in {Path(FIGURE_DIR)/comparison_name}")

 50%|█████     | 1/2 [00:00<00:00,  6.28it/s]

Calculating ARIs for n-times n-fold:  ['Code_aras_baseline', 'standard_COBRAS']
already calculated
Calculating average ARIs


100%|██████████| 2/2 [00:00<00:00,  6.09it/s]


calculating average rank
all resulting plots are in \Users\nicol\Documents\KUL 2020-2021\thesis\COBRAS+_Aras\results\results\figures\Code_aras_test_10
